# Visualize Parameter Sweep
In this notebook we reproduce the sweep summary figures, loading precomputed data to show the best median reconstruction distances that can be achieved over a large grid of diffusion constants, sequencing depths, and satellite densities.

In [ ]:
import anndata
import umap
import sys
import scanpy
import numpy as np
from matplotlib import pyplot as plt

sys.path.append("../functions/")
import alignment
import visuals

In [ ]:
# Define the parameters for the sweep
COLONIES = [25000, 50000, 100000, 250000]
SIGMA_UM = [30, 50, 75, 100]
SAMPLES = ['kidney_rpb10', 'kidney_rpb30', 'kidney_rpb50', 'kidney_rpb70', 'kidney_rpb90', 'kidney_rate003125', 
           'kidney_rate00625', 'kidney_rate0125', 'kidney_rate025', 'kidney_rate05', 'kidney_rate1']

In [ ]:
# Load the precomputed UMAPs
